In [22]:
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 
import gensim
import pandas as pd
   
# Abstract preprocessing 
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 
import gensim

def lemmatize_stemming(text):
    return ps.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result


# choose some words to be stemmed 
ps = PorterStemmer() 
words = ["change", "changes", "changing", "changed", "programers"] 
  
for w in words: 
    print(w, " : ", ps.stem(w)) 

change  :  chang
changes  :  chang
changing  :  chang
changed  :  chang
programers  :  program


In [9]:
df = pd.read_csv('sample_data.csv')

In [10]:
df.head()

,Unnamed: 0,entities,id,authors,paperAbstract,title,ft
0,1,__label__b __label__a,940a0a673ea80522074bd2e37e89b674659d26fd,"[{'name': 'Zeno Atherton', 'ids': ['91488677']...",Abstract The synthetic utility of bis(trimethy...,"Rigid rod σ-acetylide complexes of iron, ruthe...",__label__b __label__a Abstract The synthetic u...
1,3,__label__b __label__a,a090b5e520ea791acffd57706771bdc0f7ad0428,[],"Perhaps Skinner was a flamboyant cheerleader ,...",Bloom's Taxonomy of Objectives Psychology of L...,__label__b __label__a Perhaps Skinner was a fl...
2,5,__label__b __label__a,2397ef2fcfac4e4a1697e9d29164f2dd022afd73,"[{'name': 'Xu Dong', 'ids': ['145688496']}]",Based on the current situation that lots of ed...,Establishment and Significance of Comparative ...,__label__b __label__a Based on the current sit...
3,7,__label__b __label__a,58af723ae93ee59f918c5b1f76f3090fdc47cac8,"[{'name': 'Seong Heon Kim', 'ids': ['49899775'...",The gap states of the molybdenum-oxide (MoO x ...,Dark current reduction of small molecule organ...,__label__b __label__a The gap states of the mo...
4,8,__label__b __label__a,809c5a7f423502c85cf19386c99800350c728df9,"[{'name': 'Maria Hedefalk', 'ids': ['101247739...",The focus of this article is to explore presch...,Teaching for action competence,__label__b __label__a The focus of this articl...


In [11]:
# processed_docs = preprocess("Perhaps Skinner was a flamboyant cheerleader , as Leith suggested. His work, however, did have profound effects on educational thought in a number of ways related to the general concept of programmed learning. Attention was focussed first on the objectives of instruction in a general sense (Bloom et al., 1956), and then on the specific objectives of the behavioural objectives movement (Mager, 1962). The idea of mastery learning, popular in the 1930's but failing because of lack of a technology to sustain it (Block, 1971, p.4), was also resurrected as a corollary of programmed instruction, and with it evolved an interest in learning hierarchies and the structure of knowledge (Gagné and Paradise, 1961). These, in turn, have been influential in the development of systems of learning and instruction (Keller, 1968; Bloom, 1968) which have achieved the improvements in standards of learning which Skinner had hoped to bring about by his technological revolution. Here the emphasis has turned from technology in education, with its emphasis on hardware, to the technology of education. The 'Taxonomy of Educational Objectives: Handbook 1: The Cognitive Domain' (1956) arrived at a time when the demand for improvements in the efficiency of education was being made by Skinner and his colleagues. The idea for such a taxonomy came during a meeting of examiners attending the 1948 American Psychological Association Convention. Bloom and thirty three colleagues met over a five year period to discuss their taxonomy, which was then organised and written by a select committee of five members. The taxonomy was to provide a 'theoretical framework which could be used to facilitate communication among examiners.' It is intended to provide for classification of the goals of our educational system. It is expected to be of general help to all teachers, administrators, professional specialists , and research workers who deal with curricular and evaluation problems. It is especially intended to help them discuss these problems with greater precision. (Bloom et al., 1956, p.1) It was hoped that such ambiguous terms as 'really understand', 'internalize knowledge' and 'grasp the core or essence', would be redefined as a set of standard classifications, making exchange of information about curricular development and evaluation more precise. Equally important, the psychological relationships within the taxonomy were seen as forming a basis for psychological investigations to shed light on changes in the learner's behaviour. This aspect of the taxonomy has been pursued in various form")
# p = preprocess("Abstract A neutron source having an average energy of 0.532 keV has been established using a SbBe source in a polythene moderator and a B 4 C absorber assembly. The source strength ratio of the moderated to the bare source has been accurately measured using a precision long counter and agrees well with the theoretical value obtained by a Monte Carlo code.")

processed_docs = [preprocess(i) for i in df.paperAbstract]


In [26]:
ictionary = gensim.corpora.Dictionary(processed_docs)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 2, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-2):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.028*"energi" + 0.025*"object" + 0.022*"technolog" + 0.022*"educ" + 0.022*"taxonomi" + 0.022*"learn" + 0.017*"form" + 0.015*"bloom" + 0.015*"gener" + 0.015*"improv"


Topic: 1 
Words: 0.043*"establish" + 0.035*"effect" + 0.030*"sourc" + 0.026*"research" + 0.023*"time" + 0.023*"abstract" + 0.017*"work" + 0.017*"moder" + 0.017*"measur" + 0.017*"inform"




In [20]:
unseen = "Abundance, biomass, recruitment and fishing mortality of common sardine, Strangomera bentincki, and anchovy, Engraulis ringens, are estimated for stocks distributed in the central-southern area (34–40°S) off Chile. Monthly length–frequency data, from July 1990 to November 1999, were used to identify cohorts, to analyze growth, and to determine the monthly age composition of the catches. The abundance of cohorts was estimated using cohort analysis on a monthly time-scale that is sufficiently small for capturing the rapid seasonal changes that characterize both resources. An ADAPT approach was used to estimate the unknown parameters by using catch per unit effort (CPUE) data as auxiliary information. Growth was a regular process for different cohorts of both species, and a seasonal oscillating growth curve was fitted. The parameters of the seasonal von Bertalanffy growth function were: L ∞ =17.7 cm , K=0.79 yr −1 , t 0 =−0.347 yr , C=0.996, and winter point (WP)=0.373 (r 2 =0.930, n=207, P for common sardine; L ∞ =20.8 cm , K=0.444 yr −1 , t 0 =−0.392 yr , C=0.849, and WP=0.310 (r 2 =0.906, n=203, P for anchovy. The growth rates were slowest between April and May, before the spawning season. Recruitment of common sardine occurred in November, while recruitment of anchovy tended to occur in January. The yearly pulse of recruitment dominates the population dynamics of both species, and it was observed that the recruitment of the two species varies inversely. The 1997 cohort of common sardine was the lowest in the 1990s, but recruitment of anchovy was higher, probably as a consequence of the 1997–1998 El Nino event. The biomass of the common sardine stock declined from 1997 to 1999, while fishing effort was growing. On the contrary, the biomass of anchovy was higher in 1998 and 1999 due to the stronger cohorts of 1996 and 1997. Evidence for density-dependent catchability was found for sardine, but not for anchovy. At present, both species can be considered as overfished because a growing trend in fishing effort is not compensated by the productivity of the stocks."

In [19]:
bow_vector = dictionary.doc2bow(preprocess(unseen))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 2)))

Score: 0.8249324560165405	 Topic: 0.079*"chang" + 0.072*"investig"
Score: 0.02503557689487934	 Topic: 0.082*"research" + 0.062*"establish"
Score: 0.025015294551849365	 Topic: 0.094*"structur" + 0.050*"abstract"
Score: 0.025007398799061775	 Topic: 0.176*"energi" + 0.068*"object"
Score: 0.025006916373968124	 Topic: 0.084*"sourc" + 0.043*"moder"
Score: 0.025002097710967064	 Topic: 0.031*"educ" + 0.031*"taxonomi"
Score: 0.025000115856528282	 Topic: 0.046*"effect" + 0.046*"facilit"
Score: 0.025000110268592834	 Topic: 0.094*"effect" + 0.064*"select"


In [21]:
bow_vector = dictionary.doc2bow(preprocess(unseen))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 2)))

Score: 0.7811726331710815	 Topic: 0.079*"chang" + 0.072*"investig"
Score: 0.03131062909960747	 Topic: 0.082*"research" + 0.062*"establish"
Score: 0.03126268833875656	 Topic: 0.176*"energi" + 0.068*"object"
Score: 0.03125355765223503	 Topic: 0.031*"educ" + 0.031*"taxonomi"
Score: 0.03125013783574104	 Topic: 0.094*"structur" + 0.050*"abstract"
Score: 0.031250134110450745	 Topic: 0.046*"effect" + 0.046*"facilit"
Score: 0.031250130385160446	 Topic: 0.094*"effect" + 0.064*"select"
Score: 0.03125012665987015	 Topic: 0.084*"sourc" + 0.043*"moder"


In [23]:
preprocess(unseen)

['abund',
 'biomass',
 'recruit',
 'fish',
 'mortal',
 'common',
 'sardin',
 'strangomera',
 'bentincki',
 'anchovi',
 'engrauli',
 'ringen',
 'estim',
 'stock',
 'distribut',
 'central',
 'southern',
 'area',
 'chile',
 'monthli',
 'length',
 'frequenc',
 'data',
 'juli',
 'novemb',
 'identifi',
 'cohort',
 'analyz',
 'growth',
 'determin',
 'monthli',
 'composit',
 'catch',
 'abund',
 'cohort',
 'estim',
 'cohort',
 'analysi',
 'monthli',
 'time',
 'scale',
 'suffici',
 'small',
 'captur',
 'rapid',
 'season',
 'chang',
 'character',
 'resourc',
 'adapt',
 'approach',
 'estim',
 'unknown',
 'paramet',
 'catch',
 'unit',
 'effort',
 'cpue',
 'data',
 'auxiliari',
 'inform',
 'growth',
 'regular',
 'process',
 'differ',
 'cohort',
 'speci',
 'season',
 'oscil',
 'growth',
 'curv',
 'fit',
 'paramet',
 'season',
 'bertalanffi',
 'growth',
 'function',
 'winter',
 'point',
 'common',
 'sardin',
 'anchovi',
 'growth',
 'rat',
 'slowest',
 'april',
 'spawn',
 'season',
 'recruit',
 'common